**Внимание!** 

Код очень кривой, в скором времени будет отрефакторен до нормального вида 

In [1]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import numpy as np 

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Считывание данных и бинаризация категориальных признаков

In [3]:
df = pd.read_excel('movies_final_version.xlsx') 
df = df.drop(columns=['Unnamed: 0']) 
df['genre'] = df['genre'].fillna('')
df['country'] = df['country'].fillna('')

genre_encoded = df['genre'].str.get_dummies(sep=',') 
df = pd.concat([df, genre_encoded], axis=1) 
country_encoded = df['country'].str.get_dummies(sep=',') 
df = pd.concat([df, country_encoded], axis=1)

df.columns = [col.strip() for col in df.columns]
df['is_foreign'] = df['country'].apply(lambda x: 0 if 'russia' in x.lower() or 'soviet union' in x.lower() else 1)

# df['genre'] = df['genre'].apply(lambda x: tuple(x.split(',')))
df.head()

,title,year,genre,country,rating,director,Adventure,Biography,Comedy,Crime,...,Romania,Russia,Soviet Union,Spain,Sweden,United Kingdom,United States,West Germany,Yugoslavia,is_foreign
0,Разгром Японии,1945,"Documentary, War",Soviet Union,NaN,"Iosif Kheifits, Aleksandr Zarkhi",0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Близнецы,1945,Comedy,Soviet Union,6.8,Konstantin Yudin,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Весенний вальс,1940,"Comedy, Music, Musical, Romance",United States,6.7,Henry Koster,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,Аршин Мал-Алан,1945,"Comedy, Musical",Soviet Union,7.7,"Nikolay Leshchenko, Rza Takhmasib",0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Тахир и Зухра,1945,"History, Romance",Soviet Union,6.3,"Yuldash Agzamov, Nabi Ganiyev",0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [4]:
df = df.sum(axis=1, level=0)
df.head()

,title,year,genre,country,rating,director,Adventure,Biography,Comedy,Crime,...,Mexico,North Korea,Poland,Romania,Russia,Spain,Sweden,West Germany,Yugoslavia,is_foreign
0,Разгром Японии,1945,"Documentary, War",Soviet Union,0.0,"Iosif Kheifits, Aleksandr Zarkhi",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Близнецы,1945,Comedy,Soviet Union,6.8,Konstantin Yudin,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Весенний вальс,1940,"Comedy, Music, Musical, Romance",United States,6.7,Henry Koster,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,Аршин Мал-Алан,1945,"Comedy, Musical",Soviet Union,7.7,"Nikolay Leshchenko, Rza Takhmasib",0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Тахир и Зухра,1945,"History, Romance",Soviet Union,6.3,"Yuldash Agzamov, Nabi Ganiyev",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Добавляем данные о наградах 

In [5]:
df_awards = pd.read_excel('data/премии фильмов.xlsx')
df_awards.head()

,Название фильма,Регалии,Кому,Год,Источник,Заметки,Одобрение,Unnamed: 7,Unnamed: 8
0,Кутузов,сталинская премия первой степени,"режиссеру, аткерам",1946,"Искусство кино, 1946 выпуск 1",NaN,1,0.0,отрицательный отзыв
1,Иван Грозный 1 серия,сталинская премия первой степени,"режиссеру, аткерам, операторам",1946,"Искусство кино, 1946 выпуск 1",NaN,1,1.0,одобрение
2,Зоя,сталинская премия первой степени,"режиссеру, актерам, оператору",1946,"Искусство кино, 1946 выпуск 1",NaN,1,NaN,NaN
3,Георгий Саакадзе 2 серия,сталинская премия первой степени,"режиссеру, актерам, оператору",1946,"Искусство кино, 1946 выпуск 1",NaN,1,NaN,NaN
4,Радуга,сталинская премия первой степени,"режиссеру, аткерам",1946,"Искусство кино, 1946 выпуск 1",NaN,1,NaN,NaN


In [6]:
df_awards = df_awards.drop(columns=['Регалии', 'Кому', 'Год', 'Источник', 'Заметки', 'Unnamed: 7', 'Unnamed: 8']) 
df_awards.head()

,Название фильма,Одобрение
0,Кутузов,1
1,Иван Грозный 1 серия,1
2,Зоя,1
3,Георгий Саакадзе 2 серия,1
4,Радуга,1


In [7]:
df_awards['is_award'] = np.zeros(len(df_awards)) 
df_awards['is_censure'] = np.zeros(len(df_awards)) 

for i, row in df_awards.iterrows(): 
    if row['Одобрение'] == 1: 
        df_awards.loc[df_awards['Название фильма'] == row['Название фильма'], 'is_award'] = 1 
    else: 
        df_awards.loc[df_awards['Название фильма'] == row['Название фильма'], 'is_censure'] = 1 
        
        
df_awards = df_awards.drop(columns='Одобрение') 
df_awards.head()

,Название фильма,is_award,is_censure
0,Кутузов,1.0,0.0
1,Иван Грозный 1 серия,1.0,0.0
2,Зоя,1.0,0.0
3,Георгий Саакадзе 2 серия,1.0,0.0
4,Радуга,1.0,0.0


In [8]:
df_awards = df_awards.groupby('Название фильма', as_index=False).sum()

In [9]:
df_awards['is_award'] = df_awards['is_award'].apply(lambda x: min(x, 1)).astype(int)
df_awards['is_censure'] = df_awards['is_censure'].apply(lambda x: min(x, 1)).astype(int)

df_awards.head()

,Название фильма,is_award,is_censure
0,Адмирал Нахимов,1,1
1,Адмирал Ушаков,1,0
2,Академик Иван Павлов,1,0
3,Аквариум,1,0
4,Алишер Навои,1,0


In [10]:
df_awards.columns = ['title', 'is_award', 'is_censure']
df_awards.head()

,title,is_award,is_censure
0,Адмирал Нахимов,1,1
1,Адмирал Ушаков,1,0
2,Академик Иван Павлов,1,0
3,Аквариум,1,0
4,Алишер Навои,1,0


### Мерджим с наградами 

In [11]:
df['title_lower'] = df['title'].str.lower()
df_awards['title_lower'] = df_awards['title'].str.lower()

In [12]:
df = df.merge(df_awards, on='title_lower', how='left') 
df = df.drop(columns=['title_lower', 'title_y']) 
df[['is_award', 'is_censure']] = df[['is_award', 'is_censure']].fillna(0)
df

,title_x,year,genre,country,rating,director,Adventure,Biography,Comedy,Crime,...,Poland,Romania,Russia,Spain,Sweden,West Germany,Yugoslavia,is_foreign,is_award,is_censure
0,Разгром Японии,1945,"Documentary, War",Soviet Union,0.0,"Iosif Kheifits, Aleksandr Zarkhi",0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
1,Близнецы,1945,Comedy,Soviet Union,6.8,Konstantin Yudin,0,0,1,0,...,0,0,0,0,0,0,0,0,0.0,1.0
2,Весенний вальс,1940,"Comedy, Music, Musical, Romance",United States,6.7,Henry Koster,0,0,1,0,...,0,0,0,0,0,0,0,1,0.0,0.0
3,Аршин Мал-Алан,1945,"Comedy, Musical",Soviet Union,7.7,"Nikolay Leshchenko, Rza Takhmasib",0,0,1,0,...,0,0,0,0,0,0,0,0,0.0,0.0
4,Тахир и Зухра,1945,"History, Romance",Soviet Union,6.3,"Yuldash Agzamov, Nabi Ganiyev",0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,Салтанат,1955,Drama,Soviet Union,0.0,Vasili Pronin,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
813,Педагогическая поэма,1955,"Biography, Drama",Soviet Union,6.7,"Aleksey Maslyukov, Mechislava Mayevskaya",0,1,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
814,Дружба великих народов,1956,Documentary,Soviet Union,0.0,Roman Karmen,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
815,Дым в лесу,1955,Adventure,Soviet Union,6.4,"Yuriy Chulyukin, Yevgeni Karelov",1,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0


In [13]:
df.columns = ['title'] + list(df.columns[1:])
df.head()

,title,year,genre,country,rating,director,Adventure,Biography,Comedy,Crime,...,Poland,Romania,Russia,Spain,Sweden,West Germany,Yugoslavia,is_foreign,is_award,is_censure
0,Разгром Японии,1945,"Documentary, War",Soviet Union,0.0,"Iosif Kheifits, Aleksandr Zarkhi",0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
1,Близнецы,1945,Comedy,Soviet Union,6.8,Konstantin Yudin,0,0,1,0,...,0,0,0,0,0,0,0,0,0.0,1.0
2,Весенний вальс,1940,"Comedy, Music, Musical, Romance",United States,6.7,Henry Koster,0,0,1,0,...,0,0,0,0,0,0,0,1,0.0,0.0
3,Аршин Мал-Алан,1945,"Comedy, Musical",Soviet Union,7.7,"Nikolay Leshchenko, Rza Takhmasib",0,0,1,0,...,0,0,0,0,0,0,0,0,0.0,0.0
4,Тахир и Зухра,1945,"History, Romance",Soviet Union,6.3,"Yuldash Agzamov, Nabi Ganiyev",0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0


In [14]:
# df.to_excel('movies_cat_encoded.xlsx', index=False)

## Мерджим с кинотеатрами 

In [83]:
df_cinemas = pd.read_csv('data/moscow_cinemas.csv') 
df_cinemas.head()

,Cinema,First day of screening,Screening Days,Title (original),IMDB ID
0,Метрополь / Metropol',01/01/1946,50.0,Разгром Японии,171687
1,Москва / Moskva,01/01/1946,15.0,Разгром Японии,171687
2,Родина / Rodina,01/01/1946,3.0,Разгром Японии,171687
3,Востоккино / Vostokkino,01/01/1946,45.0,Разгром Японии,171687
4,Баррикады / Barrikady,01/01/1946,2.0,Разгром Японии,171687


In [84]:
df_cinemas.shape

(45944, 5)

In [85]:
df_cinemas['Cinema'] = df_cinemas['Cinema'].apply(lambda x: x.split('/')[0]) 
df_cinemas['First day of screening'] = pd.to_datetime(df_cinemas['First day of screening'], dayfirst=True) 
df_cinemas = df_cinemas.sort_values(by=['Cinema', 'First day of screening']) 
df_cinemas = df_cinemas.drop(columns='IMDB ID')

df_cinemas.head()

,Cinema,First day of screening,Screening Days,Title (original)
41,Авангард,1946-01-01,3.0,Тахир и Зухра
84,Авангард,1946-01-07,1.0,Небо Москвы
110,Авангард,1946-01-08,4.0,Серенада солнечной долины
158,Авангард,1946-01-13,2.0,Непобедимые
175,Авангард,1946-01-15,4.0,Насреддин в Бухаре


Тут считаем следующий показатель: доля дней, в которые шел хотя бы один фильм 

In [86]:
day2idx_leap = {str(day.month) + '_' + str(day.day): i for i, day in enumerate(pd.date_range(start='1/1/1948', end='31/12/1948', freq='D'))}
day2idx_normal = {str(day.month) + '_' + str(day.day): i for i, day in enumerate(pd.date_range(start='1/1/1949', end='31/12/1949', freq='D'))}

In [87]:
def fill_days(df, cinema, year): 
    if year == 1948: 
        days = 366 
        day2idx = day2idx_leap
    else: 
        days = 365
        day2idx = day2idx_normal 
        
    day_check = np.zeros(days)
    
    df_check = df[(df['Cinema'] == cinema) & 
                  ((df['First day of screening'].dt.year == year))] 
    
    for start, delta in zip(df_check['First day of screening'].values, df_check['Screening Days'].values): 
        for day in pd.date_range(start=start, end=start + pd.Timedelta(days=delta)): 
            day_check[day2idx[str(day.month) + '_' + str(day.day)]] = 1  
            
    return np.sum(day_check)
    
    
def movies_showed_ratio(df, period='first'): 
    cinemas = df['Cinema'].unique()
    ratioes = [] 
    
    if period == 'first': 
        first_year = 1948 
    else: 
        first_year = 1954
    
    for cinema in cinemas: 
        ratioes.append((fill_days(df, cinema, first_year) + fill_days(df, cinema, first_year + 1)) / 2)  
        
    return ratioes

In [20]:
df_first_period = df_cinemas[(df_cinemas['First day of screening'] >= pd.Timestamp(year=1948, month=1, day=1)) & 
                             (df_cinemas['First day of screening'] < pd.Timestamp(year=1950, month=1, day=1))
                            ]

df_first_period = df_first_period.sort_values(by='Cinema') 

df_first_period.head()

,Cinema,First day of screening,Screening Days,Title (original)
5065,Авангард,1948-01-02,4.0,Гибель орла
10738,Авангард,1949-06-01,3.0,Последний этап
10740,Авангард,1949-06-01,3.0,Пугачев
10751,Авангард,1949-06-02,1.0,Валерий Чкалов
10779,Авангард,1949-06-03,1.0,Повесть о настоящем человеке


In [88]:
df_second_period = df_cinemas[(df_cinemas['First day of screening'] >= pd.Timestamp(year=1954, month=1, day=1)) & 
                             (df_cinemas['First day of screening'] < pd.Timestamp(year=1956, month=1, day=1))
                            ]

df_second_period = df_second_period.sort_values(by='Cinema')

In [89]:
df_second_period = df_second_period.dropna(subset=['Screening Days'])

In [90]:
movies_ratio_first = movies_showed_ratio(df_first_period)
movies_ratio_second = movies_showed_ratio(df_second_period, 'second')

In [91]:
num_movies_first_period = df_first_period['Cinema'].value_counts() 
num_movies_second_period = df_second_period['Cinema'].value_counts()

In [77]:
df_merged_first_period = df_first_period.merge(df, left_on='Title (original)', right_on='title', how='left')
df_merged_first_period.head()

,Cinema,First day of screening,Screening Days,Title (original),title,year,genre,country,rating,director,...,Poland,Romania,Russia,Spain,Sweden,West Germany,Yugoslavia,is_foreign,is_award,is_censure
0,Авангард,1948-01-02,4.0,Гибель орла,Гибель орла,1941.0,Adventure,Soviet Union,5.2,Vasily Zhuravlyov,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Авангард,1949-06-01,3.0,Последний этап,Последний этап,1948.0,"Drama, History, War",Poland,7.2,Wanda Jakubowska,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,Авангард,1949-06-01,3.0,Пугачев,Пугачев,1937.0,"Biography, Drama",Soviet Union,6.5,Pavel Petrov-Bytov,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Авангард,1949-06-02,1.0,Валерий Чкалов,Валерий Чкалов,1941.0,"Drama, War",Soviet Union,6.3,Mikhail Kalatozov,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Авангард,1949-06-03,1.0,Повесть о настоящем человеке,Повесть о настоящем человеке,1948.0,"Drama, War",Soviet Union,7.0,Aleksandr Stolper,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [92]:
df_merged_second_period = df_second_period.merge(df, left_on='Title (original)', right_on='title', how='left')

In [93]:
def released_movies_before(df_movies, date):
    df_tmp = df_movies[df_movies['First day of screening'] < date] 
    movies_released_before = list(df_tmp['title'].unique()) 
    
    return movies_released_before 


def count_new_movies(df, date):
    cinema2new_list = {} 
    cinema2new_count = {}
    movies_released = set(released_movies_before(df, date))  
    
    for cinema in df['Cinema'].unique(): 
        df_cinema = df[df['Cinema'] == cinema] 
        cinema_titles = set(df_cinema['title'].unique())
        
        new_titles = list(cinema_titles - movies_released)
        cinema2new_list[cinema] = new_titles
        cinema2new_count[cinema] = len(new_titles)
        
    return cinema2new_list, cinema2new_count 

In [94]:
cinema2new_list, cinema2new_count = count_new_movies(df_merged_first_period, 
                                                     pd.Timestamp(year=1948, month=1, day=1)) 

cinema2new_list_second, cinema2new_count_second = count_new_movies(df_merged_second_period, 
                                                                  pd.Timestamp(year=1954, month=1, day=1))

In [95]:
# new_movies = set()

# for value in cinema2new_list.values(): 
#     movies = set(value) 
#     new_movies |= movies 
    
# new_movies = list(new_movies) 
# new_movies

In [96]:
# df_new_movies = df[df['title'].isin(new_movies)] 

In [97]:
# df_new_movies.to_excel('new_movies.xlsx', index=False)

### Считаем агрегированные статистики

In [30]:
df_agg_first_period = df_merged_first_period.groupby('Cinema', as_index=False).sum()
df_agg_first_period.head()

,Cinema,Screening Days,year,rating,Adventure,Biography,Comedy,Crime,Drama,Family,...,Poland,Romania,Russia,Spain,Sweden,West Germany,Yugoslavia,is_foreign,is_award,is_censure
0,Авангард,942.0,400253.0,1286.2,37.0,24.0,30.0,3.0,119.0,29.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,21.0,7.0
1,Аврора,863.0,415803.0,1358.6,29.0,22.0,51.0,3.0,124.0,12.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,28.0,15.0
2,Аквариум,415.0,188424.0,614.4,17.0,7.0,20.0,3.0,51.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,8.0,3.0
3,Арс,922.0,326290.0,1057.1,13.0,17.0,44.0,3.0,104.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,25.0,8.0
4,Баррикады,963.0,409885.0,1313.7,33.0,22.0,33.0,5.0,121.0,28.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,23.0,10.0


In [98]:
df_agg_second_period = df_merged_second_period.groupby('Cinema', as_index=False).sum()

In [30]:
df_count_movies = pd.DataFrame(num_movies_first_period).reset_index() 
df_count_movies.columns = ['Cinema', 'count_movies'] 
df_count_movies.head()

df_agg_first_period = df_agg_first_period.merge(df_count_movies, on='Cinema', how='left')

# df_agg_second_period['count_movies'] = num_movies_second_period[num_movies_second_period.keys().isin(df_agg_second_period['Cinema'].values)].values

In [99]:
df_count_movies = pd.DataFrame(num_movies_second_period).reset_index() 
df_count_movies.columns = ['Cinema', 'count_movies'] 
df_count_movies.head()

df_agg_second_period = df_agg_second_period.merge(df_count_movies, on='Cinema', how='left')

### Считаем важные статистики и приводим датасет к нормальному виду

#### Первый период (1948-1949)

In [31]:
# добавляем всякие важные фичи

# df_agg_first_period['count_movies'] = num_movies_first_period[num_movies_first_period.keys().isin(df_agg_first_period['Cinema'].values)].values
df_agg_first_period['screendays_by_year'] = movies_ratio_first

In [100]:
df_agg_second_period['screendays_by_year'] = movies_ratio_second

In [32]:
df_agg_first_period = df_agg_first_period.drop(columns=['year'])

df_agg_first_period['mean_distrib'] = df_agg_first_period['Screening Days'] / df_agg_first_period['count_movies']
df_agg_first_period['rating'] = df_agg_first_period['rating'] / df_agg_first_period['count_movies'] 

df_agg_first_period.head()

,Cinema,Screening Days,rating,Adventure,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Spain,Sweden,West Germany,Yugoslavia,is_foreign,is_award,is_censure,count_movies,screendays_by_year,mean_distrib
0,Авангард,942.0,6.066981,37.0,24.0,30.0,3.0,119.0,29.0,11.0,...,0.0,0.0,0.0,0.0,61.0,21.0,7.0,212,339.5,4.443396
1,Аврора,863.0,6.203653,29.0,22.0,51.0,3.0,124.0,12.0,7.0,...,0.0,0.0,0.0,0.0,36.0,28.0,15.0,219,318.0,3.940639
2,Аквариум,415.0,6.206061,17.0,7.0,20.0,3.0,51.0,17.0,11.0,...,0.0,0.0,0.0,0.0,26.0,8.0,3.0,99,139.5,4.191919
3,Арс,922.0,6.040571,13.0,17.0,44.0,3.0,104.0,12.0,8.0,...,0.0,0.0,0.0,0.0,27.0,25.0,8.0,175,323.5,5.268571
4,Баррикады,963.0,5.687013,33.0,22.0,33.0,5.0,121.0,28.0,11.0,...,0.0,0.0,0.0,0.0,56.0,23.0,10.0,231,337.0,4.168831


In [101]:
df_agg_second_period = df_agg_second_period.drop(columns=['year'])

df_agg_second_period['mean_distrib'] = df_agg_second_period['Screening Days'] / df_agg_second_period['count_movies']
df_agg_second_period['rating'] = df_agg_second_period['rating'] / df_agg_second_period['count_movies'] 

df_agg_second_period.head()

,Cinema,Screening Days,rating,Adventure,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Spain,Sweden,West Germany,Yugoslavia,is_foreign,is_award,is_censure,count_movies,screendays_by_year,mean_distrib
0,Авангард,1013.0,5.797318,39.0,12.0,68.0,7.0,143.0,21.0,3.0,...,0.0,0.0,1.0,0.0,86.0,10.0,5.0,261,299.5,3.881226
1,Аврора,1085.0,5.216239,34.0,11.0,94.0,10.0,158.0,21.0,6.0,...,0.0,2.0,3.0,0.0,91.0,15.0,4.0,351,314.5,3.091168
2,Аквариум,465.0,5.721849,13.0,4.0,34.0,3.0,64.0,13.0,9.0,...,0.0,0.0,3.0,1.0,42.0,7.0,0.0,119,178.5,3.907563
3,Арс,106.0,6.197222,0.0,1.0,11.0,0.0,22.0,1.0,1.0,...,0.0,0.0,0.0,0.0,12.0,4.0,0.0,36,36.5,2.944444
4,Баррикады,979.0,5.478788,27.0,6.0,58.0,6.0,115.0,22.0,8.0,...,0.0,1.0,3.0,1.0,88.0,14.0,0.0,231,329.0,4.238095


#### Добавляем данные о кинотеатрах 

In [102]:
df_cinema_moscow = pd.read_excel('data/Cinemas_Moscow_1946-1955.xlsx', header=3) 
df_cinema_moscow = df_cinema_moscow.drop(columns=['Unnamed: 4'])
df_cinema_moscow = df_cinema_moscow.drop(columns=['Location', 'Coordinates', 'Comments', 'Source'])

df_cinema_moscow.head()

,Cinema,City,Run,Seats
0,Колизей / Kolizeĭ,"Moscow, SU",1,1000
1,Метрополь / Metropol',"Moscow, SU",1,680
2,Москва / Moskva,"Moscow, SU",1,747
3,Ударник / Udarnik,"Moscow, SU",1,1587
4,Управление кинофикации Мосгорисполкома / Uprav...,NaN,NaN,NaN


In [103]:
left_bound = df_cinema_moscow[df_cinema_moscow['Cinema'] == 1954].index[0] + 1 
middle = df_cinema_moscow[df_cinema_moscow['Cinema'] == 1955].index[0] 
right_bound = df_cinema_moscow.index[-1]

In [104]:
df_1948 = df_cinema_moscow.loc[left_bound:middle-1] 
df_1948['Cinema'] = df_1948['Cinema'].apply(lambda x: x.split('/')[0])
df_1948 = df_1948.dropna(subset=df_1948.columns[1:], how='all') 
df_1948 = df_1948[df_1948['Cinema'] != 'Cinema'] 
df_1948 = df_1948.sort_values(by='Cinema')

df_1949 = df_cinema_moscow.loc[middle+1:right_bound-1] 
df_1949['Cinema'] = df_1949['Cinema'].apply(lambda x: x.split('/')[0])
df_1949 = df_1949.dropna(subset=df_1949.columns[1:], how='all')
df_1949 = df_1949[df_1949['Cinema'] != 'Cinema']
df_1949 = df_1949.sort_values(by='Cinema')

In [105]:
# ищем те кинотеатры, в которых есть изменения за год

def process_seats(x): 
    parts = str(x).split('/') 
    
    if x is np.nan: 
        return np.nan 
    
    if len(parts) == 1: 
        return int(parts[0]) 
    else: 
        return int(parts[0]) + int(parts[1]) 


df_merged = df_1948.merge(df_1949, on='Cinema', how='inner')
df_merged = df_merged.dropna() 
df_merged = df_merged.drop(columns=['City_x', 'City_y', 'Run_y'])
df_merged.columns = ['Cinema', 'rank', 'seats_1948', 'seats_1949'] 
df_merged['seats'] = df_merged['seats_1949'].apply(lambda x: process_seats(x)) 
df_merged = df_merged.drop(columns=['seats_1948', 'seats_1949'])

df_merged.head()

,Cinema,rank,seats
0,Авангард,2,463
1,Аврора,3,306
2,Аквариум,2,732
3,Баррикады,2,426
4,ВСХВ большой зал,1,736


In [106]:
df_agg_first_period = df_agg_first_period.merge(df_merged, on='Cinema', how='inner') 

In [107]:
df_agg_second_period = df_agg_second_period.merge(df_merged, on='Cinema', how='inner') 

In [38]:
df_agg_first_period['soviet_movies'] = df_agg_first_period['count_movies'] - df_agg_first_period['is_foreign'] 
df_agg_first_period

,Cinema,Screening Days,rating,Adventure,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Yugoslavia,is_foreign,is_award,is_censure,count_movies,screendays_by_year,mean_distrib,rank,seats,soviet_movies
0,Авангард,942.0,6.066981,37.0,24.0,30.0,3.0,119.0,29.0,11.0,...,0.0,61.0,21.0,7.0,212,339.5,4.443396,1,515,151.0
1,Аврора,863.0,6.203653,29.0,22.0,51.0,3.0,124.0,12.0,7.0,...,0.0,36.0,28.0,15.0,219,318.0,3.940639,3,338,183.0
2,Аквариум,415.0,6.206061,17.0,7.0,20.0,3.0,51.0,17.0,11.0,...,0.0,26.0,8.0,3.0,99,139.5,4.191919,2,732,73.0
3,Арс,922.0,6.040571,13.0,17.0,44.0,3.0,104.0,12.0,8.0,...,0.0,27.0,25.0,8.0,175,323.5,5.268571,2,273,148.0
4,Баррикады,963.0,5.687013,33.0,22.0,33.0,5.0,121.0,28.0,11.0,...,0.0,56.0,23.0,10.0,231,337.0,4.168831,2,451,175.0
5,Дзержинский ПКиО,12.0,5.557143,1.0,1.0,0.0,1.0,4.0,0.0,1.0,...,0.0,3.0,1.0,0.0,7,7.0,1.714286,2,440,4.0
6,Динамо,802.0,6.071951,14.0,16.0,30.0,4.0,97.0,9.0,6.0,...,0.0,65.0,22.0,8.0,164,331.0,4.890244,1,578,99.0
7,Диск,850.0,5.891353,43.0,28.0,46.0,5.0,141.0,34.0,17.0,...,0.0,63.0,32.0,12.0,266,312.5,3.195489,3,216,203.0
8,Заря,1014.0,6.039216,28.0,23.0,44.0,6.0,161.0,11.0,4.0,...,0.0,45.0,32.0,8.0,255,327.5,3.976471,3,263,210.0
9,Измайловский ПКиО имени Сталина,177.0,5.581159,6.0,4.0,14.0,3.0,36.0,1.0,3.0,...,0.0,23.0,4.0,1.0,69,62.0,2.565217,2,815,46.0


In [108]:
df_agg_second_period['soviet_movies'] = df_agg_second_period['count_movies'] - df_agg_second_period['is_foreign'] 
df_agg_second_period

,Cinema,Screening Days,rating,Adventure,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Yugoslavia,is_foreign,is_award,is_censure,count_movies,screendays_by_year,mean_distrib,rank,seats,soviet_movies
0,Авангард,1013.0,5.797318,39.0,12.0,68.0,7.0,143.0,21.0,3.0,...,0.0,86.0,10.0,5.0,261,299.5,3.881226,2,463,175.0
1,Аврора,1085.0,5.216239,34.0,11.0,94.0,10.0,158.0,21.0,6.0,...,0.0,91.0,15.0,4.0,351,314.5,3.091168,3,306,260.0
2,Аквариум,465.0,5.721849,13.0,4.0,34.0,3.0,64.0,13.0,9.0,...,1.0,42.0,7.0,0.0,119,178.5,3.907563,2,732,77.0
3,Баррикады,979.0,5.478788,27.0,6.0,58.0,6.0,115.0,22.0,8.0,...,1.0,88.0,14.0,0.0,231,329.0,4.238095,2,426,143.0
4,ВСХВ малый зал,6.0,3.725000,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,0.0,0.0,4,2.0,1.500000,2,438,3.0
5,Дзержинский ПКиО,4.0,6.275000,1.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,4,2.0,1.000000,2,652,2.0
6,Динамо,832.0,5.743069,30.0,10.0,40.0,8.0,120.0,18.0,5.0,...,4.0,81.0,10.0,1.0,202,326.0,4.118812,1,522,121.0
7,Диск,990.0,5.146450,39.0,13.0,71.0,10.0,164.0,23.0,5.0,...,2.0,89.0,13.0,3.0,338,325.0,2.928994,3,207,249.0
8,Дружба,707.0,5.755495,29.0,7.0,44.0,6.0,100.0,24.0,6.0,...,0.0,75.0,10.0,0.0,182,276.0,3.884615,1,277,107.0
9,Заря,1043.0,5.464262,49.0,11.0,68.0,13.0,157.0,17.0,5.0,...,2.0,93.0,18.0,5.0,305,307.5,3.419672,3,228,212.0


In [43]:
cnt = pd.DataFrame({
    'Cinema': list(cinema2new_count.keys()), 
    'new_movies': list(cinema2new_count.values())
})
cnt.columns = ['Cinema', 'new_movies'] 
cnt.head()

,Cinema,new_movies
0,Авангард,152
1,Аврора,143
2,Аквариум,86
3,Арс,125
4,Баррикады,161


In [109]:
cnt = pd.DataFrame({
    'Cinema': list(cinema2new_count_second.keys()), 
    'new_movies': list(cinema2new_count_second.values())
})
cnt.columns = ['Cinema', 'new_movies'] 
cnt.head()

,Cinema,new_movies
0,Авангард,191
1,Аврора,215
2,Аквариум,98
3,Арс,30
4,Баррикады,168


In [44]:
df_agg_first_period = df_agg_first_period.merge(cnt, on='Cinema', how='left')

In [110]:
df_agg_second_period = df_agg_second_period.merge(cnt, on='Cinema', how='left')

In [57]:
df_agg_first_period.to_excel('agg_stat_first_period.xlsx', index=False) 

In [46]:
df_agg_second_period

,Cinema,Screening Days,rating,Adventure,Biography,Comedy,Crime,Drama,Family,Fantasy,...,is_foreign,is_award,is_censure,count_movies,screendays_by_year,mean_distrib,rank,seats,soviet_movies,new_movies
0,Авангард,1013.0,5.797318,39.0,12.0,68.0,7.0,143.0,21.0,3.0,...,86.0,10.0,5.0,261,0.0,3.881226,2,463,175.0,191
1,Аврора,1085.0,5.216239,34.0,11.0,94.0,10.0,158.0,21.0,6.0,...,91.0,15.0,4.0,351,0.0,3.091168,3,306,260.0,215
2,Аквариум,465.0,5.721849,13.0,4.0,34.0,3.0,64.0,13.0,9.0,...,42.0,7.0,0.0,119,0.0,3.907563,2,732,77.0,98
3,Баррикады,979.0,5.478788,27.0,6.0,58.0,6.0,115.0,22.0,8.0,...,88.0,14.0,0.0,231,0.0,4.238095,2,426,143.0,168
4,ВСХВ малый зал,6.0,3.725000,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,1.0,0.0,0.0,4,0.0,1.500000,2,438,3.0,3
5,Дзержинский ПКиО,4.0,6.275000,1.0,1.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,4,0.0,1.000000,2,652,2.0,4
6,Динамо,832.0,5.743069,30.0,10.0,40.0,8.0,120.0,18.0,5.0,...,81.0,10.0,1.0,202,0.0,4.118812,1,522,121.0,171
7,Диск,990.0,5.146450,39.0,13.0,71.0,10.0,164.0,23.0,5.0,...,89.0,13.0,3.0,338,0.0,2.928994,3,207,249.0,215
8,Дружба,707.0,5.755495,29.0,7.0,44.0,6.0,100.0,24.0,6.0,...,75.0,10.0,0.0,182,0.0,3.884615,1,277,107.0,144
9,Заря,1043.0,5.464262,49.0,11.0,68.0,13.0,157.0,17.0,5.0,...,93.0,18.0,5.0,305,0.0,3.419672,3,228,212.0,212


In [111]:
df_agg_second_period.to_excel('agg_stat_second_period.xlsx', index=False) 